In [1]:
#working

In [140]:

import os
import random
import time
from pydantic import BaseModel, Field
from typing import Dict, Any, TypedDict, Literal, List, Optional

from langchain_groq import ChatGroq
from langgraph.graph import StateGraph, END
from langchain.agents import AgentExecutor
from langchain_core.messages import SystemMessage, HumanMessage
from langchain_core.runnables import RunnableParallel
from langchain_core.tools import tool
from langchain_core.prompts import PromptTemplate
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langgraph.prebuilt import create_react_agent

In [141]:
from dotenv import load_dotenv
load_dotenv()
groq_api_key = os.getenv("GROQ_API_KEY")



In [142]:
# --- Data Storage ---
# This class simulates a database or key-value store.
# In production, this could be replaced with Redis, a file system, or a database.
class DataStorage:
    """Simulates a database. In production, replace with Redis, a DB, etc."""
    _storage = {
        'document_content': {},
        'summary_output': {},
        'insights_output': {},
        'action_items': {}
    }
    
    @classmethod
    def store(cls, data_type: str, data: any) -> str:
        """Store data and return a unique reference ID."""
        # Using a simple timestamp and random number for a unique ID
        uid = f"{data_type}_{int(time.time() * 1000)}_{random.randint(100, 999)}"
        cls._storage[data_type][uid] = data
        print(f"📦 Stored data of type '{data_type}' with ID: {uid}")
        return uid
    
    @classmethod
    def retrieve(cls, data_type: str, uid: str) -> any:
        """Retrieve data by its reference ID."""
        print(f" retrievel data of type '{data_type}' with ID: {uid}")
        return cls._storage[data_type].get(uid)



In [164]:

# --- Refactored GraphState ---
# The state now holds lightweight IDs instead of large text blobs.
class GraphState(TypedDict):
    """Refactored state using IDs for scalability."""
    document_content_id: str
    summary_status: str
    insights_status: str
    summary_id: Optional[str]
    insights_id: Optional[str]
    action_items_id: Optional[str]
    iteration: int
    error_message: str
    current_reasoning: str
    next: str


In [165]:
# Action Item Extraction Tool 
class ActionItem(BaseModel):
    task: str = Field(description="The specific action or task to be completed.")
    owner: Optional[str] = Field(description="The person or team responsible for the task.")
    deadline: Optional[str] = Field(description="The due date for the task, e.g., 'EOW', '2024-08-15'.")

In [166]:


# Simple but intelligent decision model
class SupervisorDecision(BaseModel):
    """AI Supervisor decision with intelligent reasoning"""
    next_action: Literal[
        "call_both_parallel", 
        "call_summary_only", 
        "call_insights_only", 
        "end_workflow"
    ] = Field(description="What to do next in the workflow")
    
    reasoning: str = Field(description="Why this decision makes sense for the workflow goal")
    
    confidence: float = Field(
        description="Confidence in this decision (0.0 to 1.0)", 
        ge=0.0, le=1.0
    )

In [167]:

@tool("extract_and_store_action_items", return_direct=True)
def extract_and_store_action_items(document_content: str) -> str:
    """
    Identifies action items in a document, structures them,
    stores them, and returns a confirmation with the storage ID.
    """
    print("\n🛠️ Action Item Tool Called...")

    prompt = ChatPromptTemplate.from_messages([
        ("system", "You are an expert at extracting structured data. Identify all action items from the text. For each, extract the task, owner, and deadline. Use 'N/A' if missing."),
        ("human", "Extract action items from this document:\n\n---\n\n{document}")
    ])
    llm = ChatGroq(temperature=0, model="deepseek-r1-distill-llama-70b")

    # # Use with_structured_output to get clean JSON
    # extractor = prompt | llm.with_structured_output(schema=List[ActionItem])

    # This correctly defines the schema for a list of items
    class ActionItems(BaseModel):
        action_items: List[ActionItem]

    extractor = prompt | llm.with_structured_output(schema=ActionItems)

    try:
        # The result will be an instance of the ActionItems class
        result = extractor.invoke({"document": document_content})

        action_items_list = result.action_items if result and hasattr(result, "action_items") else []
        
        # if not result.action_items:
        #     return "No action items were found in the document."

        if not action_items_list:
            return "No action items were found in the document."

        # Store the list of Pydantic objects
        storage_id = DataStorage.store('action_items', action_items_list)
        print(f"✅ Action items extracted and stored successfully.")
        
        # Return a structured confirmation message
        return f"Confirmation: Successfully stored {len(action_items_list)} action items with ID {storage_id}"

    except Exception as e:
        print(f"❌ Error in Action Item Tool: {e}")
        return "An error occurred during action item extraction."

In [ ]:


def intelligent_supervisor(state: GraphState) ->  Dict[str, Any]:
    """
    Smart supervisor that understands the workflow goal:
    - Get meeting minutes summary 
    - Get key insights in parallel
    - Be intelligent about when to retry vs when to finish
    """
    print(f"\n🧠 AI Supervisor thinking... (Iteration: {state['iteration']})")
    
    # Enhanced ReAct-style system prompt
    system_prompt = """
    You are an intelligent workflow supervisor using ReAct (Reasoning + Acting) methodology. 

    ## YOUR MISSION:
    Ensure we successfully get a meeting summary and key insights stored efficiently.

    ## ReAct PROCESS - Think step by step:
    **THOUGHT**: First, analyze the current workflow state. What's working? What failed? Why might it have failed?
    **OBSERVATION**: A task is 'successful' if its status is 'success' AND its ID exists. A task has 'failed' if its status is 'failed'.
    **ACTION**: Based on your thought and observation, decide the smartest next action.

    ## CURRENT WORKFLOW STATE:
    - Summary Status: {summary_status} (summary_id: {summary_id})
    - Insights Status: {insights_status} (insights_id: {insights_id})
    - Action Items Found: {action_items_count}
    - Current Iteration: {iteration}
    - Last Error: {error_msg}

    ## AVAILABLE ACTIONS:
    - call_both_parallel: Run both agents simultaneously.
    - call_summary_only: Focus only on getting the summary.
    - call_insights_only: Focus only on getting insights.  
    - end_workflow: Use this ONLY when both Summary and Insights have a 'success' status and their IDs exist.

    ## INTELLIGENT DECISION GUIDELINES:
    - Iteration 1: Usually start with parallel execution for efficiency.
    - One success, one failure: Target retry on the failed agent only.
    - Both failed in early iterations: Likely a temporary issue, retry both.
    - Multiple failures: Consider if we should accept partial results or end the workflow.
    - Both successful: Mission accomplished! Time to end the workflow.

    **Remember**: You're optimizing for getting useful meeting analysis (summary + insights), not just perfect success rates.

    Use the ReAct process: THOUGHT → OBSERVATION → ACTION with clear reasoning.
    """

    print("state:", state)
    
     # FIX: The formatting logic now uses the correct 'summary_id' and 'insights_id' keys from the state.
    formatted_prompt = system_prompt.format(
        summary_status=state['summary_status'],
        summary_id=f"'{state['summary_id']}'" if state.get('summary_id') else "None",
        insights_status=state['insights_status'],
        insights_id=f"'{state['insights_id']}'" if state.get('insights_id') else "None",
        # action_items_count=len(state.get('action_items_id')),
        iteration=state['iteration'],
        error_msg=state.get('error_message') or "None"
    )
    
    
    # Setup Groq API with Qwen model
    groq_api_key = os.getenv("GROQ_API_KEY")
    if not groq_api_key:
        raise ValueError("GROQ_API_KEY environment variable is not set")
    
    os.environ["GROQ_API_KEY"] = groq_api_key
    
    llm = ChatGroq(
        temperature=0,
        model="deepseek-r1-distill-llama-70b"  
    )
    
    messages = [
        SystemMessage(content=formatted_prompt),
        HumanMessage(content="Use ReAct methodology: THOUGHT → OBSERVATION → ACTION. Analyze the workflow state and decide what to do next for our goal. Think step by step.")
    ]
    
    # Get structured decision from LLM
    try:
        decision = llm.with_structured_output(SupervisorDecision).invoke(messages)
    except Exception as e:
        print(f"⚠️ LLM call failed: {e}")
        # Fallback to simulation if LLM fails
        decision = simulate_smart_decision(state)
    
    # # Simulate intelligent decision (replace with real LLM)
    # decision = simulate_smart_decision(state)
    
    print(f"💭 AI Reasoning: {decision.reasoning}")
    print(f"⚡ Decision: {decision.next_action} (confidence: {decision.confidence:.2f})")
    
    # # Update state with AI reasoning
    # new_state = state.copy()
    # new_state['next'] = decision.next_action  # Store the next action
    # new_state['current_reasoning'] = decision.reasoning
    
    # return decision.next_action if decision.next_action != "end_workflow" else END
    goto = decision.next_action if decision.next_action != "end_workflow" else END

    print('goto:',goto)

    return {
        "current_reasoning": decision.reasoning,
        "next": decision.next_action # Store the determined next action
    }
        

In [149]:


def simulate_smart_decision(state: GraphState) -> SupervisorDecision:
    """
    Simulate what an intelligent model like Qwen would decide
    This is just simulation - replace with actual LLM call
    """
    
    summary_status = state['summary_status']
    insights_status = state['insights_status']
    iteration = state['iteration']
    
    # Smart decision making that a good reasoning model would do
    
    # Goal achieved - both successful
    if summary_status == "success" and insights_status == "success":
        return SupervisorDecision(
            next_action="end_workflow",
            reasoning="Perfect! Both summary and insights are ready. Our workflow goal is complete - we have meeting summary + key insights as requested.",
            confidence=1.0
        )
    
    # First iteration - start efficiently  
    if iteration == 1 and summary_status == "pending" and insights_status == "pending":
        return SupervisorDecision(
            next_action="call_both_parallel", 
            reasoning="First attempt - running both summary and insights agents in parallel for efficiency. This is the optimal starting strategy.",
            confidence=0.9
        )
    
    # One succeeded, one failed - targeted retry
    if summary_status == "success" and insights_status == "failed":
        if iteration <= 3:  # Smart about retry limits
            return SupervisorDecision(
                next_action="call_insights_only",
                reasoning="Summary is ready, but insights failed. Retrying only insights agent since summary is already successful. Efficient targeted approach.",
                confidence=0.8
            )
        else:
            return SupervisorDecision(
                next_action="end_workflow",
                reasoning="Summary is ready and we've tried insights multiple times. Sometimes partial success is acceptable for meeting processing workflow.",
                confidence=0.6
            )
    
    if insights_status == "success" and summary_status == "failed":
        if iteration <= 3:
            return SupervisorDecision(
                next_action="call_summary_only", 
                reasoning="Insights are ready, but summary failed. Retrying only summary agent since insights are already successful. Focused retry approach.",
                confidence=0.8
            )
        else:
            return SupervisorDecision(
                next_action="end_workflow",
                reasoning="Insights are ready and we've tried summary multiple times. We have key insights from the meeting which provides value.",
                confidence=0.6
            )
    
    # Both failed - intelligent retry decision
    if summary_status == "failed" and insights_status == "failed":
        if iteration <= 2:
            return SupervisorDecision(
                next_action="call_both_parallel",
                reasoning="Both agents failed, but it's early in the process. Likely a temporary issue (network/API). Retrying both in parallel - efficient recovery approach.",
                confidence=0.7
            )
        elif iteration <= 4:
            return SupervisorDecision(
                next_action="call_both_parallel", 
                reasoning="Multiple failures but still within reasonable retry range. The meeting document seems valid, so this might be temporary service issues. One more parallel attempt.",
                confidence=0.5
            )
        else:
            return SupervisorDecision(
                next_action="end_workflow",
                reasoning="After multiple attempts, continuing may not be productive. This could be a deeper issue with the document format or service availability. Ending workflow.",
                confidence=0.8
            )
    
    # Default intelligent fallback
    return SupervisorDecision(
        next_action="call_both_parallel",
        reasoning="Current state requires both agents to run. Taking parallel approach for efficiency in meeting processing workflow.", 
        confidence=0.6
    )


In [150]:
def route_supervisor_decision(state: GraphState) -> str:
    """
    Routes the workflow based on the decision stored by the intelligent supervisor.
    """
    return state['next'] 

In [151]:
# def run_summary_agent(state: GraphState) -> dict:
#     """
#     An agentic node that intelligently generates a meeting summary.
#     It has a specific persona, instructions, and robust error handling.
#     """
#     print("\n🤖 Agentic Summary Node Called...")

#     # 1. Define the Agent's Persona and Mission via a System Prompt
#     system_prompt = """
#     You are an expert Meeting Summarization Agent. Your sole mission is to create a concise, structured, and insightful summary from the provided meeting minutes.
#     If you identify any specific tasks assigned to people (action items), Do include the action items directly in your final summary. Also, you MUST use the 'extract_and_store_action_items' tool to process them. you can mention that action items were noted and processed separately.
    
#     You have access to the following tools to help you:
#     {tools}
    
#     Tool Names: {tool_names}

#     ## Your Guidelines:
#     1.  **Identify Core Content**: Focus on extracting the most critical information:
#         - **Key Decisions Made**: What was formally decided?
#         - **Action Items**: What are the specific next steps? Who is responsible (owner)? What are the deadlines?
#         - **Major Topics Discussed**: Briefly mention the main subjects of conversation.
#         - **Outcomes & Resolutions**: What was the final result of the discussions?

#     2.  **Prioritize Significance**: Do not just list topics in order. Your value is in identifying what truly matters. A brief 2-minute decision that sets the company's direction is more important than a 30-minute unresolved debate.

#     3.  **Structure the Output**: Present the summary in a clean, professional format using Markdown. Use headings (#), subheadings (##), and bullet points (-) for clarity.

#     4.  **Be Objective**: Summarize what was said and decided without adding your own opinions or interpretations. Stick to the facts presented in the document.

#     Your final output should be ONLY the structured summary, ready to be shared with meeting attendees.
#     """

#     # 2. Setup the Agent with proper prompt template
#     prompt = ChatPromptTemplate.from_messages([
#         ("system", system_prompt),
#         ("user", "{input}"),
#         MessagesPlaceholder(variable_name="agent_scratchpad"),
#     ])
#     prompt = ChatPromptTemplate.from_messages([
#         ("system", """
#         You are an expert Meeting Summarization Agent. Your sole mission is to create a concise, structured, and insightful summary from the provided meeting minutes.
#         Prioritize key decisions, major topics, and outcomes.
#         If you identify any specific tasks assigned to people (action items), you MUST use the 'extract_and_store_action_items' tool to process them.
#         Do not list the action items in your final answer; just mention that they were processed separately.

#         You have access to the following tools:
#         {tools}

#         To use a tool, respond with a JSON blob with 'action' and 'action_input' keys. The 'action' must be one of [{tool_names}].
#         When you have your final summary, you MUST respond with a JSON blob with a single 'answer' key.
#         """),
#         ("user", "{input}"),
#         MessagesPlaceholder(variable_name="agent_scratchpad"),
#     ])
   

#     groq_api_key = os.getenv("GROQ_API_KEY")
#     if not groq_api_key:
#         raise ValueError("GROQ_API_KEY environment variable is not set")

#     llm = ChatGroq(temperature=0, model="qwen/qwen3-32b")

#     # The agent uses the LLM and prompt, with tools for action item extraction
#     tools = [extract_and_store_action_items]
#     agent = create_react_agent(llm, tools, prompt=prompt)
#     summary_agent = AgentExecutor(agent=agent, tools=tools, verbose=True)

#     # 3. Invoke the Agent with Error Handling
#     try:
#         document_content = DataStorage.retrieve('document_content', state['document_content_id'])
#         if not document_content:
#             raise ValueError("Failed to retrieve document content from storage.")
        
#         print("🧠 Agent is thinking and generating the summary...")
#         # The input to the agent is the document content from DataStorage
#         # This allows the agent to work with a lightweight reference ID instead of large text blobs.
#         # This is more scalable and efficient for large documents.
#         agent_input = {
#             "input": f"Please generate a summary for the following meeting minutes:\n\n---\n\n{document_content}",
#             "document_content": document_content 
#         }
#         result = summary_agent.invoke(agent_input)
        

#         # The agent's final answer is in the output key for ReAct agents
#         generated_summary = result.get('output', result['messages'][-1].content if 'messages' in result else str(result))
#         print("✅ Summary Agent: Generated summary successfully.")
        
#         # Store the generated summary
#         summary_id = DataStorage.store('summary_output', generated_summary)
        
#         return {
#             "summary_status": "success",
#             "summary_id": summary_id,
#             "error_message": ""
#         }

#     except Exception as e:
#         print(f"❌ Error in Agentic Summary Node: {e}")
        
#         # If the agent fails, report the failure back to the graph state
#         return {
#             "summary_status": "failed",
#             "error_message": f"Summary Agent Error: {str(e)}"
#         }

In [ ]:
# Make sure to have the correct imports
from langchain_groq import ChatGroq
from langgraph.prebuilt import create_react_agent # Modern agent builder
from langchain_core.messages import SystemMessage, ToolMessage
import os
import re

# Assuming GraphState and DataStorage are defined elsewhere, as in your original code
# and the tool 'extract_and_store_action_items' is also defined.

def run_summary_agent(state: GraphState) -> dict:
    """
    An agentic node that intelligently generates a meeting summary using the
    modern langgraph.prebuilt.create_react_agent.
    """
    print("\n🤖 Modern Agentic Summary Node Called...")

    # 1. Define the Agent's Persona and Mission
    # This prompt is simpler because modern tool-calling models don't need
    # explicit instructions on JSON formatting.
    system_prompt = """You are an expert Meeting Summarization Agent. Your sole mission is to create a concise, structured, and insightful summary from the provided meeting minutes.

## Your Guidelines:
1.  **Core Content**: Focus on key decisions, major topics discussed, and final outcomes.
2.  **Tool Use**: If you identify any specific tasks or action items, you MUST use the `extract_and_store_action_items` tool to process them.
3.  **Final Output**: In your final answer, do not list the action items . Simply state that they were identified and processed. The final output should ONLY be the clean, Markdown-formatted summary.
"""

    # 2. Setup the LLM, Tools, and Agent
    try:
        groq_api_key = os.getenv("GROQ_API_KEY")
        if not groq_api_key:
            raise ValueError("GROQ_API_KEY environment variable is not set")

        # Note: Using a powerful model like Llama 3 70b is recommended for complex summarization.
        # "qwen/qwen3-32b" is not a standard model on the Groq API.
        llm = ChatGroq(temperature=0, model="qwen/qwen3-32b")
        tools = [extract_and_store_action_items]

        # This single line creates the modern, compiled agent graph.
        # It replaces both the legacy create_react_agent and the AgentExecutor.
        summary_agent_graph = create_react_agent(llm, tools, prompt=system_prompt)

    except Exception as e:
        print(f"❌ Error during agent setup: {e}")
        return {
            "summary_status": "failed",
            "error_message": f"Summary Agent Setup Error: {str(e)}"
        }

    # 3. Invoke the Agent with Error Handling
    try:
        document_content = DataStorage.retrieve('document_content', state['document_content_id'])
        if not document_content:
            raise ValueError("Failed to retrieve document content from storage.")

        print("🧠 Agent is thinking and generating the summary...")

        # The input for a langgraph agent is a dictionary with a "messages" key.
        agent_input = {
            "messages": [
                ("user", f"Please generate a summary for the following meeting minutes:\n\n---\n\n{document_content}")
            ]
        }
        result = summary_agent_graph.invoke(agent_input)

        # The agent's final answer is the content of the last message in the state.
        generated_summary = result["messages"][-1].content
        print("✅ Summary Agent: Generated summary successfully.:::", generated_summary)

        # action_item_id = None
        # for message in result.get("messages", []):
        #     # Find the ToolMessage from our specific tool
        #     if isinstance(message, ToolMessage) and "Confirmation: Successfully stored" in message.content:
        #         # Parse the ID from the confirmation string
        #         match = re.search(r'ID (\S+)', message.content)
        #         if match:
        #             action_item_id = match.group(1)
        #             print(f"✅ Summary node captured Action Item ID: {action_item_id}")
        #             break
        

        # Store the generated summary
        summary_id = DataStorage.store('summary_output', generated_summary)

        return {
            "summary_status": "success",
            "summary_id": summary_id,
            "error_message": ""
        }

    except Exception as e:
        print(f"❌ Error in Agentic Summary Node: {e}")
        return {
            "summary_status": "failed",
            "error_message": f"Summary Agent Error: {str(e)}"
        }

In [154]:
# Make sure to have the correct imports
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
import os

# Assuming GraphState and DataStorage are defined elsewhere

def run_insights_agent(state: GraphState) -> dict:
    """
    A specialist node that uses a focused LLM chain to extract deep,
    strategic insights from meeting minutes.
    """
    print("\n💡 Specialist Insights Node Called...")

    # 1. Crafting a High-Fidelity Prompt for the Insights Specialist
    # This is the heart of the "agentic" behavior for a no-tool task.
    # We give it a strong persona, a clear mission, and a structured framework.
    prompt_template = ChatPromptTemplate.from_template(
        """
        # PERSONA & MISSION
        You are a premier Business Strategy Analyst and Insights Specialist. You are not a summarizer; you are a sense-maker. Your mission is to transcend the surface-level details of the provided meeting minutes and distill them into high-level, actionable, strategic insights for executive review. You must uncover the 'why' behind the 'what'.

        # ANALYTICAL FRAMEWORK
        Read the entire document first. Then, apply the following framework to generate your insights. Do not mention this framework in your output; use it as your internal guide.

        1.  **Identify Key Themes**: What are the recurring strategic ideas, concerns, or opportunities being discussed? Look for patterns, not just topics.
            - *Example: "A recurring theme was the tension between innovation speed and maintaining product quality."*

        2.  **Analyze Critical Decisions & Implications**: For each major decision, state it concisely and then, most importantly, explain its strategic implication.
            - *Example: "Decision: The 'Phoenix Project' was greenlit. Implication: This signals a major strategic pivot for the company, deprioritizing legacy systems to capture a new market segment."*

        3.  **Surface Actionable Insights**: What can the leadership team learn from the conversation? These are not action items (tasks for individuals), but strategic recommendations for the team or company as a whole.
            - *Example: "Insight: The extended debate over resource allocation for Q4 reveals a potential misalignment on departmental priorities. A cross-departmental priority-setting workshop is recommended."*

        # OUTPUT REQUIREMENTS
        - Your final output must be ONLY the structured insights.
        - Use clean, professional Markdown formatting (headings, subheadings, bullet points).
        - Do NOT summarize the meeting. Do NOT mention action items. Your focus is exclusively on strategic insights.
        - Begin your analysis directly without any preamble like "Here are the insights...".

        # DOCUMENT FOR ANALYSIS
        ---
        {document_content}
        ---
                """
            )

    # 2. Setup a simple, powerful LLM Chain (No agent needed)
    try:
        groq_api_key = os.getenv("GROQ_API_KEY")
        if not groq_api_key:
            raise ValueError("GROQ_API_KEY environment variable is not set")

        # A powerful model is essential for this kind of deep reasoning task.
        llm = ChatGroq(temperature=0.2, model="llama3-70b-8192")

        # This simple LCEL chain is more efficient than an agent for no-tool tasks.
        insights_chain = prompt_template | llm | StrOutputParser()

    except Exception as e:
        print(f"❌ Error during insights chain setup: {e}")
        return {
            "insights_status": "failed",
            "error_message": f"Insights Chain Setup Error: {str(e)}"
        }

    # 3. Invoke the Chain with Error Handling
    try:
        document_content = DataStorage.retrieve('document_content', state['document_content_id'])
        if not document_content:
            raise ValueError("Failed to retrieve document content from storage.")

        print("🧠 Specialist is analyzing and extracting insights...")
        
        # Invoke the chain directly with the document content.
        generated_insights = insights_chain.invoke({"document_content": document_content})

        print("✅ Insights Specialist: Extracted insights successfully.", generated_insights)

        # Store the generated insights
        insights_id = DataStorage.store('insights_output', generated_insights)

        return {
            "insights_status": "success",
            "insights_id": insights_id,
            "error_message": ""
        }

    except Exception as e:
        print(f"❌ Error in Specialist Insights Node: {e}")
        return {
            "insights_status": "failed",
            "error_message": f"Insights Specialist Error: {str(e)}"
        }

In [155]:
def run_both_parallel_agents(state: GraphState) -> dict:
    """Runs the refactored summary and insights agents in parallel."""
    print("\n---RUNNING BOTH REFACTORED AGENTS IN PARALLEL---")

    parallel_runnable = RunnableParallel(
        summary_result=run_summary_agent,
        insights_result=run_insights_agent
    )
    parallel_results = parallel_runnable.invoke(state)
    
    # This combination logic remains the same and works with the new outputs
    return {**parallel_results['summary_result'], **parallel_results['insights_result']}

In [156]:
def increment_iteration(state: GraphState) -> GraphState:
    """Track iterations for intelligent decision making"""
    new_state = state.copy()
    new_state['iteration'] += 1
    print(f"\n--- Workflow Iteration: {new_state['iteration']} ---")
    return new_state

In [157]:
# Build the workflow graph
workflow = StateGraph(GraphState)

# Add nodes
workflow.add_node("increment_iteration", increment_iteration)
workflow.add_node("intelligent_supervisor", intelligent_supervisor)
workflow.add_node("run_summary_agent", run_summary_agent)
workflow.add_node("run_insights_agent", run_insights_agent)
workflow.add_node("run_parallel_agents", run_both_parallel_agents)

In [158]:

# Set entry point
workflow.set_entry_point("increment_iteration")

# Define initial edge from entry point to supervisor
workflow.add_edge("increment_iteration", "intelligent_supervisor")

In [159]:
# The AI supervisor makes intelligent routing decisions
# - The edge starts from the 'intelligent_supervisor' node.
# - The 'route_supervisor_decision' function defines how to choose the next path.
workflow.add_conditional_edges(
    "intelligent_supervisor",     # Conditional edges originate from the 'intelligent_supervisor' node
    route_supervisor_decision,    # This function defines the routing logic (returns the key for the dict)
    {
        "call_both_parallel": "run_parallel_agents", # Map decision to the parallel execution node
        "call_insights_only": "run_insights_agent",  # Map decision to the insights agent node
        "call_summary_only": "run_summary_agent",    # Map decision to the summary agent node
        "end_workflow": END                          # Map decision to the END state
    }
)


In [160]:
# After each agent completes, return to the supervisor for the next intelligent decision
workflow.add_edge("run_summary_agent", "intelligent_supervisor")
workflow.add_edge("run_insights_agent", "intelligent_supervisor")
workflow.add_edge("run_parallel_agents", "intelligent_supervisor") # Add this edge for parallel execution


In [161]:
# Compile the workflow
app = workflow.compile()


In [169]:
# --- Test the intelligent workflow ---
print("🚀 Starting Intelligent Meeting Processing Workflow")

    # A. Store the initial document and get its ID
document_text = "Meeting Minutes - July 22, 2025\nAttendees: Alice, Bob, Charlie\n1. Project Phoenix: Bob confirmed the server is deployed. Action Item: Charlie to schedule a budget review by Friday.\n2. Marketing: The 'Summer Sale' will launch on August 1st. Action Item: AI team to refine the BERT chatbot for user intent by July 28th."
doc_id = DataStorage.store('document_content', document_text)

    # B. Define the initial state using the ID
initial_state = {
        "document_content_id": doc_id,
        "summary_status": "pending",  
        "insights_status": "pending", 
        "summary_id": None,
        "insights_id": None,
        "action_items_id": None,
        "iteration": 0,
        "error_message": None,
        "current_reasoning": "",
        "next": ""
    }

# In your main execution script:

# C. Run the workflow and capture the final state
final_state = {}
for step in app.stream(initial_state, {"recursion_limit": 10}):
    print("\n" + "="*40)
    step_name = list(step.keys())[0]
    print(f"STEP: {step_name}")
    print(f"STATE: {step[step_name]}")
    # Instead of overwriting, UPDATE the dictionary to build the complete final state
    final_state.update(step[step_name])

    # C. Run the workflow and capture the final state
# final_state = {}
# for step in app.stream(initial_state, {"recursion_limit": 10}):
#     print("\n" + "="*40)
#     print(f"STEP: {list(step.keys())[0]}")
#     print(f"STATE: {step[list(step.keys())[0]]}")
#     final_state = step[list(step.keys())[0]]

print("\n🎯 Workflow completed with AI supervision!")

    # D. Retrieve and display the final results from DataStorage
# D. Retrieve and display the final results from DataStorage
print("\n\n--- FINAL RESULTS ---")

print("\n✅ Final State:",final_state)

# Corrected to use 'summary_id' and 'insights_id'
summary = DataStorage.retrieve('summary_output', final_state.get('summary_id', ''))
insights = DataStorage.retrieve('insights_output', final_state.get('insights_id', ''))

print("\n✅ Final Summary:")
print(summary or "Not generated.")

print("\n✅ Final Insights:")
print(insights or "Not generated.")

print("\n✅ Extracted Action Items:")

for item_id in final_state.get('action_items_ids', []):
    action_items = DataStorage.retrieve('action_items', item_id)
    if action_items:
        # Assuming action_items is a list of Pydantic models or dicts
        for i, item in enumerate(action_items):
            owner = getattr(item, 'owner', 'N/A')
            task = getattr(item, 'task', 'N/A')
            deadline = getattr(item, 'deadline', 'N/A')
            print(f"  - Task: {task}, Owner: {owner}, Deadline: {deadline}")
    else:
        print("  - Could not retrieve action items.")

🚀 Starting Intelligent Meeting Processing Workflow
📦 Stored data of type 'document_content' with ID: document_content_1753356097298_475

--- Workflow Iteration: 1 ---

STEP: increment_iteration
STATE: {'document_content_id': 'document_content_1753356097298_475', 'summary_status': 'pending', 'insights_status': 'pending', 'summary_id': None, 'insights_id': None, 'iteration': 1, 'error_message': None, 'current_reasoning': '', 'next': ''}

🧠 AI Supervisor thinking... (Iteration: 1)
state: {'document_content_id': 'document_content_1753356097298_475', 'summary_status': 'pending', 'insights_status': 'pending', 'summary_id': None, 'insights_id': None, 'iteration': 1, 'error_message': None, 'current_reasoning': '', 'next': ''}
💭 AI Reasoning: Starting with parallel execution in the first iteration is efficient as both tasks are pending and there are no errors.
⚡ Decision: call_both_parallel (confidence: 0.95)
goto: call_both_parallel

STEP: intelligent_supervisor
STATE: {'current_reasoning': 'S

In [170]:

# Define the graph
workflow = StateGraph(GraphState)
workflow.add_node("increment_iteration", increment_iteration)
workflow.add_node("intelligent_supervisor", intelligent_supervisor)
workflow.add_node("run_summary_agent", run_summary_agent)
workflow.add_node("run_insights_agent", run_insights_agent)
workflow.add_node("run_parallel_agents", run_both_parallel_agents)

# Define the edges
workflow.set_entry_point("increment_iteration")
workflow.add_edge("increment_iteration", "intelligent_supervisor")
workflow.add_conditional_edges(
    "intelligent_supervisor", route_supervisor_decision,
    {"call_both_parallel": "run_parallel_agents", "call_insights_only": "run_insights_agent",
        "call_summary_only": "run_summary_agent", "end_workflow": END}
)
workflow.add_edge("run_summary_agent", "increment_iteration")
workflow.add_edge("run_insights_agent", "increment_iteration")
workflow.add_edge("run_parallel_agents", "increment_iteration")

app = workflow.compile()

print("🚀 Starting Intelligent Meeting Processing Workflow")
document_text = "Meeting Minutes - July 23, 2025\nAttendees: Alice, Bob, Charlie\n1. Project Phoenix: Bob confirmed the server is deployed. Action Item: Charlie to schedule a budget review by Friday.\n2. Marketing: The 'Summer Sale' will launch on August 1st. Action Item: AI team to refine the BERT chatbot for user intent by July 28th."
doc_id = DataStorage.store('document_content', document_text)

initial_state = {
    "document_content_id": doc_id,
    "summary_status": "pending",
    "insights_status": "pending",
    "summary_id": None,
    "insights_id": None,
    "action_items_ids": [],
    "iteration": 0,
    "error_message": "",
    "current_reasoning": "",
    "next": ""
}

final_state = {}
for step in app.stream(initial_state, {"recursion_limit": 15}):
    node_name = list(step.keys())[0]
    state_update = step[node_name]
    print("\n" + "="*40 + f"\nSTEP: {node_name}\n" + "="*40)
    if isinstance(state_update, dict):
        final_state.update(state_update)
    print(f"STATE UPDATE: {state_update}")

print("\n\n🎯 Workflow completed!")

# FIX: Corrected the data_type for retrieving the final results.
summary = DataStorage.retrieve('summary_output', final_state.get('summary_id', ''))
insights = DataStorage.retrieve('insights_output', final_state.get('insights_id', ''))
print("\n✅ Final Summary:\n", summary or "Not generated.")
print("\n✅ Final Insights:\n", insights or "Not generated.")

print("\n✅ Extracted Action Items:")
action_item_ids = final_state.get('action_items_ids', [])
if not action_item_ids:
    print("  - No action items were processed.")
else:
    for item_id in action_item_ids:
        action_items = DataStorage.retrieve('action_items', item_id)
        if action_items:
            for i, item in enumerate(action_items):
                print(f"  - Task: {item.task}, Owner: {item.owner}, Deadline: {item.deadline}")
        else:
            print(f"  - Could not retrieve action items for ID: {item_id}")


🚀 Starting Intelligent Meeting Processing Workflow
📦 Stored data of type 'document_content' with ID: document_content_1753356176890_893

--- Workflow Iteration: 1 ---

STEP: increment_iteration
STATE UPDATE: {'document_content_id': 'document_content_1753356176890_893', 'summary_status': 'pending', 'insights_status': 'pending', 'summary_id': None, 'insights_id': None, 'iteration': 1, 'error_message': '', 'current_reasoning': '', 'next': ''}

🧠 AI Supervisor thinking... (Iteration: 1)
state: {'document_content_id': 'document_content_1753356176890_893', 'summary_status': 'pending', 'insights_status': 'pending', 'summary_id': None, 'insights_id': None, 'iteration': 1, 'error_message': '', 'current_reasoning': '', 'next': ''}
💭 AI Reasoning: Starting with parallel execution in the first iteration is efficient as both tasks are pending and there are no errors.
⚡ Decision: call_both_parallel (confidence: 0.95)
goto: call_both_parallel

STEP: intelligent_supervisor
STATE UPDATE: {'current_reas

In [ ]:
# import os
# import random
# import re
# import time
# from typing import TypedDict, Literal, List, Optional, Dict, Any

# from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
# from langchain_core.pydantic_v1 import BaseModel, Field
# from langchain.agents import create_react_agent, AgentExecutor
# from langchain_core.tools import tool
# from langchain_groq import ChatGroq
# from langgraph.graph import StateGraph, END

# # --- 1. Data Storage and State Definition ---

# class DataStorage:
#     """Simulates a database. In production, replace with Redis, a DB, etc."""
#     _storage = {
#         'document_content': {},
#         'summary_output': {},
#         'insights_output': {},
#         'action_items': {}
#     }
    
#     @classmethod
#     def store(cls, data_type: str, data: any) -> str:
#         uid = f"{data_type}_{int(time.time() * 1000)}_{random.randint(100, 999)}"
#         cls._storage[data_type][uid] = data
#         print(f"📦 Stored '{data_type}' with ID: {uid}")
#         return uid
    
#     @classmethod
#     def retrieve(cls, data_type: str, uid: str) -> any:
#         print(f"📥 Retrieving '{data_type}' with ID: {uid}")
#         return cls._storage[data_type].get(uid)

# class GraphState(TypedDict):
#     """State using the correct ID field names for scalability."""
#     document_content_id: str
#     summary_status: str
#     insights_status: str
#     summary_id: Optional[str]
#     insights_id: Optional[str]
#     action_items_ids: List[str]
#     iteration: int
#     error_message: str
#     current_reasoning: str
#     next: str

# # --- 2. Tool for Action Item Extraction ---

# class ActionItem(BaseModel):
#     task: str = Field(description="The specific action or task to be completed.")
#     owner: str = Field(description="The person or team responsible for the task.")
#     deadline: str = Field(description="The due date for the task, e.g., 'EOW', '2024-08-15'.")

# @tool
# def extract_and_store_action_items(document_content: str) -> str:
#     """
#     Identifies action items in a document, structures them,
#     stores them, and returns a confirmation with the storage ID.
#     """
#     print("\n🛠️ Action Item Tool Called...")
#     prompt = ChatPromptTemplate.from_messages([
#         ("system", "You are an expert at extracting structured data. Identify all action items from the provided text. For each action item, extract the task description, the owner, and the deadline. Use 'N/A' if a detail is missing."),
#         ("human", "Please extract all action items from this document:\n\n---\n\n{document}")
#     ])
#     llm = ChatGroq(temperature=0, model="llama3-70b-8192")
#     extractor = prompt | llm.with_structured_output(schema=List[ActionItem])
    
#     try:
#         action_items_list = extractor.invoke({"document": document_content})
#         if not action_items_list:
#             return "No action items were found in the document."
#         storage_id = DataStorage.store('action_items', action_items_list)
#         print(f"✅ Action items stored successfully.")
#         return f"Successfully extracted and stored {len(action_items_list)} action items with ID: {storage_id}"
#     except Exception as e:
#         return f"An error occurred while trying to extract action items: {e}"

# # --- 3. Refactored Agent Nodes (with Corrected Prompts) ---

# def run_summary_agent(state: GraphState) -> dict:
#     """Refactored summary agent using the correct ChatPromptTemplate structure."""
#     print("\n🤖 Refactored Summary Agent Called...")
    
#     # FIX: Using ChatPromptTemplate and providing explicit JSON output instructions.
#     prompt = ChatPromptTemplate.from_messages([
#         ("system", """
#         You are an expert Meeting Summarization Agent. Your sole mission is to create a concise, structured, and insightful summary from the provided meeting minutes.
#         Prioritize key decisions, major topics, and outcomes.
#         If you identify any specific tasks assigned to people (action items), you MUST use the 'extract_and_store_action_items' tool to process them.
#         Do not list the action items in your final answer; just mention that they were processed separately.

#         You have access to the following tools:
#         {tools}

#         To use a tool, respond with a JSON blob with 'action' and 'action_input' keys.
#         When you have your final summary, you MUST respond with a JSON blob with a single 'answer' key.
#         """),
#         ("user", "{input}"),
#         MessagesPlaceholder(variable_name="agent_scratchpad"),
#     ])
    
#     llm = ChatGroq(temperature=0, model="llama3-70b-8192")
#     tools = [extract_and_store_action_items]
#     agent = create_react_agent(llm, tools, prompt=prompt)
#     agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, handle_parsing_errors=True)

#     try:
#         document_content = DataStorage.retrieve('document_content', state['document_content_id'])
#         if not document_content:
#             raise ValueError("Failed to retrieve document content from storage.")

#         agent_input = {
#             "input": f"Please generate a summary for the following meeting minutes:\n\n---\n\n{document_content}",
#             # Pass the document content to the tool as well, as it needs it directly
#             "document_content": document_content 
#         }
#         result = agent_executor.invoke(agent_input)
        
#         summary_id = DataStorage.store('summary_output', result['output'])
        
#         new_action_item_ids = state.get('action_items_ids', [])
#         if 'intermediate_steps' in result:
#             for _, tool_output in result['intermediate_steps']:
#                 match = re.search(r"ID: (action_items_\d+_\d+)", tool_output)
#                 if match:
#                     new_action_item_ids.append(match.group(1))

#         return {
#             "summary_status": "success",
#             "summary_id": summary_id,
#             "action_items_ids": new_action_item_ids,
#             "error_message": ""
#         }
#     except Exception as e:
#         print(f"❌ Error in Summary Agent: {e}")
#         return {
#             "summary_status": "failed",
#             "error_message": f"Summary Agent Error: {str(e)}"
#         }

# def run_insights_agent(state: GraphState) -> dict:
#     """Refactored insights agent using the correct ChatPromptTemplate structure."""
#     print("\n🤖 Refactored Insights Agent Called...")
    
#     # FIX: Using ChatPromptTemplate and providing explicit JSON output instructions.
#     prompt = ChatPromptTemplate.from_messages([
#         ("system", """
#         You are an expert Meeting Insights Agent. Your mission is to extract not just facts, but the underlying meaning, implications of decisions, and strategic insights from the meeting minutes.
#         You are focused on the 'why' and 'so what', not just the 'what'. You do not need to process action items.

#         Since you don't need to use tools for this task, think through the user's request and then provide your final answer.
#         When you have your final answer, you MUST respond with a JSON blob with a single 'answer' key.
#         """),
#         ("user", "{input}"),
#         MessagesPlaceholder(variable_name="agent_scratchpad"),
#     ])
    
#     llm = ChatGroq(temperature=0.1, model="llama3-70b-8192")
#     tools = [] 
#     agent = create_react_agent(llm, tools, prompt=prompt)
#     agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, handle_parsing_errors=True)

#     try:
#         document_content = DataStorage.retrieve('document_content', state['document_content_id'])
#         if not document_content:
#             raise ValueError("Failed to retrieve document content from storage.")

#         agent_input = {
#             "input": f"Please extract key insights from these meeting minutes:\n\n---\n\n{document_content}"
#         }
#         result = agent_executor.invoke(agent_input)
        
#         insights_id = DataStorage.store('insights_output', result['output'])
        
#         return {
#             "insights_status": "success",
#             "insights_id": insights_id,
#             "error_message": ""
#         }
#     except Exception as e:
#         print(f"❌ Error in Insights Agent: {e}")
#         return {
#             "insights_status": "failed",
#             "error_message": f"Insights Agent Error: {str(e)}"
#         }

# def run_both_parallel_agents(state: GraphState) -> dict:
#     """Runs the refactored summary and insights agents in parallel."""
#     print("\n---RUNNING BOTH REFACTORED AGENTS IN PARALLEL---")
#     # Note: LangGraph's parallel execution might not show verbose agent logs clearly.
#     # Running agents sequentially is better for debugging agent thoughts.
#     return {**run_summary_agent(state), **run_insights_agent(state)}

# # --- 4. Supervisor and Routing Logic ---

# class SupervisorDecision(BaseModel):
#     """AI Supervisor's decision with reasoning."""
#     next_action: Literal["call_both_parallel", "call_summary_only", "call_insights_only", "end_workflow"]
#     reasoning: str

# def intelligent_supervisor(state: GraphState) -> Dict[str, Any]:
#     """Smart supervisor with the heuristic-based prompt."""
#     print(f"\n🧠 AI Supervisor thinking... (Iteration: {state['iteration']})")
    
#     system_prompt = """
#     You are an intelligent workflow supervisor using ReAct (Reasoning + Acting) methodology. 
#     Your mission is to ensure we successfully get a meeting summary and key insights stored efficiently.
#     A task is 'successful' if its status is 'success' AND its ID exists. A task has 'failed' if its status is 'failed'.
#     Based on your observation of the state, decide the smartest next action.

#     ## CURRENT WORKFLOW STATE:
#     - Summary Status: {summary_status} (summary_id: {summary_id})
#     - Insights Status: {insights_status} (insights_id: {insights_id})
#     - Iteration: {iteration}
#     - Last Error: {error_msg}

#     ## AVAILABLE ACTIONS:
#     - call_both_parallel, call_summary_only, call_insights_only, end_workflow

#     ## INTELLIGENT DECISION GUIDELINES:
#     - Iteration 1: Usually start with parallel execution for efficiency.
#     - One success, one failure: Target retry on the failed agent only.
#     - Both failed in early iterations: Likely a temporary issue, retry both.
#     - Both successful: Mission accomplished! Time to end the workflow.
#     """
#     formatted_prompt = system_prompt.format(
#         summary_status=state['summary_status'],
#         summary_id=f"'{state['summary_id']}'" if state.get('summary_id') else "None",
#         insights_status=state['insights_status'],
#         insights_id=f"'{state['insights_id']}'" if state.get('insights_id') else "None",
#         iteration=state['iteration'],
#         error_msg=state.get('error_message') or "None"
#     )
    
#     llm = ChatGroq(temperature=0, model="llama3-70b-8192")
#     decision_chain = ChatPromptTemplate.from_messages(
#         [("system", formatted_prompt), ("human", "Analyze the state and decide the next action.")]
#     ) | llm.with_structured_output(SupervisorDecision)
    
#     try:
#         decision = decision_chain.invoke({})
#     except Exception as e:
#         return {"next": "end_workflow", "current_reasoning": f"Supervisor LLM failed: {e}"}

#     print(f"💭 AI Reasoning: {decision.reasoning}\n⚡ Decision: {decision.next_action}")
    
#     if state['summary_status'] == 'success' and state['insights_status'] == 'success':
#         return {"next": "end_workflow"}

#     return {"current_reasoning": decision.reasoning, "next": decision.next_action}

# def route_supervisor_decision(state: GraphState) -> str:
#     return state['next']

# def increment_iteration(state: GraphState) -> dict:
#     return {"iteration": state.get('iteration', 0) + 1}

# # --- 5. Graph Definition and Execution ---

# if __name__ == '__main__':
#     # Define the graph
#     workflow = StateGraph(GraphState)
#     workflow.add_node("increment_iteration", increment_iteration)
#     workflow.add_node("intelligent_supervisor", intelligent_supervisor)
#     workflow.add_node("run_summary_agent", run_summary_agent)
#     workflow.add_node("run_insights_agent", run_insights_agent)
#     workflow.add_node("run_parallel_agents", run_both_parallel_agents)

#     # Define the edges
#     workflow.set_entry_point("increment_iteration")
#     workflow.add_edge("increment_iteration", "intelligent_supervisor")
#     workflow.add_conditional_edges(
#         "intelligent_supervisor", route_supervisor_decision,
#         {"call_both_parallel": "run_parallel_agents", "call_insights_only": "run_insights_agent",
#          "call_summary_only": "run_summary_agent", "end_workflow": END}
#     )
#     workflow.add_edge("run_summary_agent", "increment_iteration")
#     workflow.add_edge("run_insights_agent", "increment_iteration")
#     workflow.add_edge("run_parallel_agents", "increment_iteration")
    
#     app = workflow.compile()

#     print("🚀 Starting Intelligent Meeting Processing Workflow")
#     document_text = "Meeting Minutes - July 23, 2025\nAttendees: Alice, Bob, Charlie\n1. Project Phoenix: Bob confirmed the server is deployed. Action Item: Charlie to schedule a budget review by Friday.\n2. Marketing: The 'Summer Sale' will launch on August 1st. Action Item: AI team to refine the BERT chatbot for user intent by July 28th."
#     doc_id = DataStorage.store('document_content', document_text)

#     initial_state = {
#         "document_content_id": doc_id,
#         "summary_status": "pending",
#         "insights_status": "pending",
#         "summary_id": None,
#         "insights_id": None,
#         "action_items_ids": [],
#         "iteration": 0,
#         "error_message": "",
#         "current_reasoning": "",
#         "next": ""
#     }

#     final_state = {}
#     for step in app.stream(initial_state, {"recursion_limit": 15}):
#         node_name = list(step.keys())[0]
#         state_update = step[node_name]
#         print("\n" + "="*40 + f"\nSTEP: {node_name}\n" + "="*40)
#         if isinstance(state_update, dict):
#             final_state.update(state_update)
#         print(f"STATE UPDATE: {state_update}")

#     print("\n\n🎯 Workflow completed!")
#     summary = DataStorage.retrieve('summary_output', final_state.get('summary_id', ''))
#     insights = DataStorage.retrieve('insights_output', final_state.get('insights_id', ''))
#     print("\n✅ Final Summary:\n", summary or "Not generated.")
#     print("\n✅ Final Insights:\n", insights or "Not generated.")
    
#     print("\n✅ Extracted Action Items:")
#     action_item_ids = final_state.get('action_items_ids', [])
#     if not action_item_ids:
#         print("  - No action items were processed.")
#     else:
#         for item_id in action_item_ids:
#             action_items = DataStorage.retrieve('action_items', item_id)
#             if action_items:
#                 for i, item in enumerate(action_items):
#                     print(f"  - Task: {item.task}, Owner: {item.owner}, Deadline: {item.deadline}")
#             else:
#                 print(f"  - Could not retrieve action items for ID: {item_id}")


In [ ]:
# if __name__ == '__main__':
# #     # This is a simple test to show how the node works in isolation
#     test_state = GraphState(
#         document_content="Meeting Minutes - July 21, 2025\nAttendees: Alice, Bob, Charlie\n1. Project Phoenix Update: Bob confirmed the new server is deployed. Alice raised a concern about the budget overrun. It was decided to review the Q3 budget next week. Action Item: Charlie to schedule a budget review meeting by Friday, July 25th.\n2. Marketing Campaign: Discussed the new 'Summer Sale' campaign. It will launch on August 1st. All assets are ready.",
#         summary_status="pending",
#         insights_status="pending",
#         summary_output="",
#         insights_output="",
#         iteration=0,
#         error_message="",
#         current_reasoning="",
#         next=""
#     )

#     result_update = run_summary_agent_agentic(test_state)
#     print("\n--- TEST RESULT ---")
#     import json
#     print(json.dumps(result_update, indent=2))

In [ ]:
# def run_summary_agent(state: GraphState) -> dict:
#     """
#     An agentic node that intelligently generates a meeting summary.
#     It has a specific persona, instructions, and robust error handling.
#     """
#     print("\n🤖 Agentic Summary Node Called...")

#     # 1. Define the Agent's Persona and Mission via a System Prompt
#     system_prompt = """
#     You are an expert Meeting Summarization Agent. Your sole mission is to create a concise, structured, and insightful summary from the provided meeting minutes.
#     If you identify any specific tasks assigned to people (action items), Do include the action items directly in your final summary. Also, you MUST use the 'extract_and_store_action_items' tool to process them. you can mention that action items were noted and processed separately.

#     ## Your Guidelines:
#     1.  **Identify Core Content**: Focus on extracting the most critical information:
#         - **Key Decisions Made**: What was formally decided?
#         - **Action Items**: What are the specific next steps? Who is responsible (owner)? What are the deadlines?
#         - **Major Topics Discussed**: Briefly mention the main subjects of conversation.
#         - **Outcomes & Resolutions**: What was the final result of the discussions?

#     2.  **Prioritize Significance**: Do not just list topics in order. Your value is in identifying what truly matters. A brief 2-minute decision that sets the company's direction is more important than a 30-minute unresolved debate.

#     3.  **Structure the Output**: Present the summary in a clean, professional format using Markdown. Use headings (#), subheadings (##), and bullet points (-) for clarity.

#     4.  **Be Objective**: Summarize what was said and decided without adding your own opinions or interpretations. Stick to the facts presented in the document.

#     Your final output should be ONLY the structured summary, ready to be shared with meeting attendees.
#     """

#     # 2. Setup the Agent using the working pattern from your previous code
#     prompt = ChatPromptTemplate.from_messages([
#         ("system", system_prompt),
#         MessagesPlaceholder(variable_name="messages"),
#     ])

#     groq_api_key = os.getenv("GROQ_API_KEY")
#     if not groq_api_key:
#         raise ValueError("GROQ_API_KEY environment variable is not set")

#     llm = ChatGroq(temperature=0, model="qwen/qwen3-32b")

#     # Use the same pattern as your working code
#     tools = [extract_and_store_action_items]
#     agent = create_react_agent(
#         model=llm,  # Using model parameter like your working code
#         tools=tools,
#         prompt=prompt
#     )

#     # 3. Invoke the Agent with Error Handling
#     try:
#         document_content = DataStorage.retrieve('document_content', state['document_content_id'])
#         if not document_content:
#             raise ValueError("Failed to retrieve document content from storage.")
        
#         print("🧠 Agent is thinking and generating the summary...")
        
#         # Create messages in the format the agent expects
#         messages = [HumanMessage(content=f"Please generate a summary for the following meeting minutes:\n\n---\n\n{document_content}")]
        
#         # Invoke using the same pattern as your working code
#         result = agent.invoke({"messages": messages})

#         # Extract the generated summary
#         generated_summary = result['messages'][-1].content
#         print("✅ Summary Agent: Generated summary successfully.")
        
#         # Store the generated summary
#         summary_id = DataStorage.store('summary_output', generated_summary)
        
#         return {
#             "summary_status": "success",
#             "summary_id": summary_id,
#             "error_message": ""
#         }

#     except Exception as e:
#         print(f"❌ Error in Agentic Summary Node: {e}")
        
#         # If the agent fails, report the failure back to the graph state
#         return {
#             "summary_status": "failed",
#             "error_message": f"Summary Agent Error: {str(e)}"
#         }


# def run_insights_agent(state: GraphState) -> dict:
#     """
#     An agentic node that generates intelligent insights from meeting minutes.
#     """
#     print("\n🤖 Agentic Insights Node Called...")

#     # 1. Define the Agent's Persona and Mission via a System Prompt
#     system_prompt = """
#     You are an expert Meeting Insights Agent. Your mission is to analyze meeting minutes and extract meaningful patterns, strategic implications, and actionable intelligence.

#     ## Your Analysis Framework:
#     1. **Strategic Insights**: What are the long-term implications of the decisions made?
#     2. **Pattern Recognition**: Are there recurring themes, concerns, or opportunities?
#     3. **Risk Assessment**: What potential challenges or obstacles were identified?
#     4. **Stakeholder Analysis**: Who are the key players and what are their positions?
#     5. **Follow-up Opportunities**: What questions remain unanswered or need further exploration?

#     ## Output Guidelines:
#     - Present insights in a structured, analytical format using Markdown
#     - Focus on what's NOT immediately obvious from a simple reading
#     - Provide context and implications, not just facts
#     - Be strategic and forward-thinking in your analysis
#     - Use professional language suitable for executive briefings

#     Your final output should be a comprehensive insights report ready for leadership review.
#     """

#     # 2. Setup the Agent using the working pattern
#     prompt = ChatPromptTemplate.from_messages([
#         ("system", system_prompt),
#         MessagesPlaceholder(variable_name="messages"),
#     ])

#     groq_api_key = os.getenv("GROQ_API_KEY")
#     if not groq_api_key:
#         raise ValueError("GROQ_API_KEY environment variable is not set")

#     llm = ChatGroq(temperature=0.1, model="llama3-70b-8192")

#     # This agent doesn't need tools, keeping it focused on analysis
#     insights_agent = create_react_agent(
#         model=llm,
#         tools=[],  # No tools needed for insights
#         prompt=prompt
#     )

#     # 3. Invoke the Agent with Error Handling
#     try:
#         document_content = DataStorage.retrieve('document_content', state['document_content_id'])
#         if not document_content:
#             raise ValueError("Failed to retrieve document content from storage.")
        
#         print("🧠 Insights Agent is analyzing the meeting...")
        
#         # Create messages in the format the agent expects
#         messages = [HumanMessage(content=f"Please analyze and generate insights for the following meeting minutes:\n\n---\n\n{document_content}")]
        
#         # Invoke the agent
#         result = insights_agent.invoke({"messages": messages})

#         # Extract the generated insights
#         generated_insights = result['messages'][-1].content
#         print("✅ Insights Agent: Generated insights successfully.")
        
#         # Store the generated insights
#         insights_id = DataStorage.store('insights_output', generated_insights)
        
#         return {
#             "insights_status": "success",
#             "insights_id": insights_id,
#             "error_message": ""
#         }

#     except Exception as e:
#         print(f"❌ Error in Agentic Insights Node: {e}")
        
#         return {
#             "insights_status": "failed",
#             "error_message": f"Insights Agent Error: {str(e)}"
#         }

In [66]:
#fake ones

In [68]:

# # Keep your original agent functions - they're fine
# def run_summary_agent(state: GraphState) -> GraphState:
#     """Summary agent - focused on meeting summary"""
#     print(f"\n📝 Summary Agent working on meeting minutes...")
#     new_state = state.copy()
    
#     # Simulate work with some intelligence about content
#     success_rate = 0.1 if "meeting" in state['document_content'].lower() else 0.3
    
#     if random.random() < success_rate:
#         new_state['summary_status'] = "success"
#         new_state['summary_output'] = f"Meeting Summary: Key decisions and action items extracted from meeting minutes (iteration {state['iteration']})"
#         print("✅ Summary: Generated meeting summary successfully")
#     else:
#         new_state['summary_status'] = "failed" 
#         new_state['summary_output'] = "Summary generation failed"
#         new_state['error_message'] = "Temporary API issue during summary generation"
#         print("❌ Summary: Failed (likely temporary issue)")
    
#     time.sleep(1)
#     return new_state



In [ ]:

# def run_insights_agent(state: GraphState) -> dict:
#     """
#     Insights agent - focused on meeting insights.
#     FIX: Now returns a dictionary and clears the error message on success.
#     """
#     print(f"\n🔍 Insights Agent extracting key insights...")
    
#     # Simulate work
#     success_rate = 0.6 if "meeting" in state['document_content'].lower() else 0.3
#     time.sleep(1)

#     if random.random() < success_rate:
#         print("✅ Insights: Extracted meeting insights successfully")
#         # On success, return a success status AND an empty error message
#         return {
#             "insights_status": "success",
#             "insights_output": f"Meeting Insights: Key themes and decisions identified (iteration {state['iteration']})",
#             "error_message": ""
#         }
#     else:
#         print("❌ Insights: Failed (simulating temporary issue)")
#         # On failure, return a failed status AND a descriptive error message
#         return {
#             "insights_status": "failed",
#             "insights_output": "Insights generation failed",
#             "error_message": "Temporary processing issue during insights extraction"
#         }

In [ ]:
# from langchain_core.runnables import RunnableParallel

# def run_both_parallel_agents(state: GraphState) -> dict:
#     """
#     Runs summary and insights agents in parallel.
#     This now calls both of the new agentic nodes.
#     """
#     print("\n---RUNNING BOTH AGENTIC NODES IN PARALLEL---")

#     parallel_runnable = RunnableParallel(
#         summary_result=run_summary_agent,
#         insights_result=run_insights_agent 
#     )

#     parallel_results = parallel_runnable.invoke(state)

#     summary_updates = parallel_results['summary_result']
#     insights_updates = parallel_results['insights_result']

#     combined_updates = {**summary_updates, **insights_updates}

#     summary_error = summary_updates.get("error_message", "")
#     insights_error = insights_updates.get("error_message", "")

#     if summary_error and insights_error:
#         combined_updates["error_message"] = f"Summary Error: {summary_error} | Insights Error: {insights_error}"
#     elif summary_error:
#         combined_updates["error_message"] = summary_error
#     elif insights_error:
#         combined_updates["error_message"] = insights_error
#     else:
#         combined_updates["error_message"] = ""

#     return combined_updates